In [2]:
import json
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [6]:
results_dir = "../../synth_results/asymp_exp copy"

experiment_dirs = [
    os.path.join(results_dir, d)
    for d in os.listdir(results_dir)
    if os.path.isdir(os.path.join(results_dir, d)) and d.startswith("experiment_")
]

for exp_dir in tqdm(experiment_dirs, desc="Processing experiment directories"):

    config_path = os.path.join(exp_dir, "config.json")
    if not os.path.exists(config_path):
        print(f"Config file not found in {exp_dir}. Skipping...")
        continue
    
    with open(config_path, "r") as f:
        config = json.load(f)
    ng = config.get("normalize_gini")
    n  = config.get("normalize")

    # decide what the new normalize_gini should be
    if ng is None and n is None:
        new_ng = None
    elif ng is not None and n is None:
        new_ng = ng
    elif ng is None and n is not None:
        new_ng = n
    else:
        # this “shouldn’t” happen, but we’ll record it so you can spot it
        new_ng = f"ERROR: both normalize_gini={ng!r} and normalize={n!r} present"

    new_config = {
        "data": {
            "n_samples_train": config.get("n_samples_train", None),
            "n_samples_test":  config.get("n_samples_test",  None),
            "batch_size_train": config.get("batch_size_train", None),
            "batch_size_test":  config.get("batch_size_test",  None),
            "seed_train":       config.get("seed_train",       None),
            "seed_test":        config.get("seed_test",        None)
        },
        "clustering": {
            "name":        config.get("method",            None),
            "n_clusters":  config.get("n_clusters",        None),
            "seed":        config.get("kmeans_seed",       None),
            "init_scheme": config.get("init_scheme",       None),
            "space":       config.get("partionning_space", None),
            "cov_type":    config.get("cov_type",          None),
            "temperature": config.get("temperature",       None), 
            "normalize_gini": new_ng,
        },
        "experiment": {
            "datetime": config.get("experiment_datetime", None),
            "folder":   config.get("experiment_folder",   None)
        }
    }
    backup_path = os.path.join(exp_dir, "config.json.bak")
    try:
        os.replace(config_path, backup_path)
        with open(config_path, "w") as f:
            json.dump(new_config, f, indent=4)
    except Exception as e:
        print(f"Failed to update {config_path}: {e}")
        continue
    

Processing experiment directories: 100%|██████████| 4819/4819 [01:33<00:00, 51.44it/s]
